In [1]:
import pandas
import numpy as np
import tensorflow as tf
import os
import math
import random as rand
from random import random

# Data Preparation

In [2]:
trainDataFile = 'ijcnlp_dailydialog/train/dialogues_train.txt'
trainLabelsFile = 'ijcnlp_dailydialog/train/dialogues_act_train.txt' 

In [3]:
testDataFile = 'ijcnlp_dailydialog/validation/dialogues_validation.txt'
testLablesFile = 'ijcnlp_dailydialog/validation/dialogues_act_validation.txt'

In [4]:
lineSeparator = '__eou__'

In [5]:
def getData(dialogueFile, labelFile):
    with open(dialogueFile) as f:
        lines = f.readlines() 
    with open(labelFile) as f:
        labels = f.readlines()    
    
    inputs = []
    inputs_all = []
    for line in lines:
        line = line.rstrip()
        line = line.split(lineSeparator)
        inputs.append(line[:-1])
        inputs_all.extend(line[:-1])
    
    acts = []
    acts_all = []
    for label in labels:
        label = label.rstrip()
        label = label.split()
        label = np.array(list(map(int, label))) - 1
        acts.append(label)
        acts_all.extend(label)
        
    return (inputs_all, inputs, acts_all, acts)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [7]:
count_vect = CountVectorizer()

In [8]:
trainX_all, trainX_data, trainY_all, trainY = getData(trainDataFile, trainLabelsFile)

In [9]:
trainX_all = count_vect.fit_transform(trainX_all)

In [10]:
trainX = []
for i in range(len(trainX_data)):
    trainX.append(count_vect.transform(trainX_data[i]))

In [11]:
trainX[0].shape

(10, 17635)

In [12]:
valX_all, valX_data, valY_all, valY = getData(testDataFile, testLablesFile)

In [13]:
valX = []
for i in range(len(valX_data)):
    valX.append(count_vect.transform(valX_data[i]))

In [14]:
valX_all = count_vect.transform(valX_all)

# Train a classifier

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier

In [16]:
model1 = LogisticRegression(n_jobs = -1)
model2 = RandomForestClassifier(n_estimators = 100)
model3 = LinearSVC()
model3 = CalibratedClassifierCV(model3)

In [17]:
model1.fit(trainX_all, trainY_all)
model2.fit(trainX_all, trainY_all)
model3.fit(trainX_all, trainY_all)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv=3, method='sigmoid')

In [ ]:
model3

In [18]:
probs1 = model1.predict_proba(valX_all)
probs2 = model2.predict_proba(valX_all)
probs3 = model3.predict_proba(valX_all)

In [ ]:
probs3[0]

In [19]:
# Parameters used.
MODEL_PATH = './model_crf_v1.ckpt'

In [20]:
def get_sequence_length(x_data):
    seq = [];
    for i in range(len(x_data)):
        seq.append(x_data[i].shape[0])
    return seq

In [21]:
x_train = trainX
y_train = trainY

x_test = valX
y_test = valY

sequence_length_train = get_sequence_length(x_train)
sequence_length_test = get_sequence_length(x_test)


In [ ]:
sequence_length_train[4]

In [ ]:
len(x_train)
x_train[0].shape

In [22]:
x_train[2].shape
x_train_new = []
for r in x_train:
    print(r.shape)
    p1 = model1.predict_proba(r)
#     print(p1.shape)
    p2 = model2.predict_proba(r)
    res = np.concatenate((p1, p2), 1)
    p3 = model3.predict_proba(r)
    res = np.concatenate((res, p3), 1)
    x_train_new.append(res)
x_train = x_train_new

(10, 17635)
(6, 17635)
(5, 17635)
(4, 17635)
(9, 17635)
(10, 17635)
(7, 17635)
(13, 17635)
(12, 17635)
(6, 17635)
(8, 17635)
(5, 17635)
(8, 17635)
(4, 17635)
(15, 17635)
(7, 17635)
(12, 17635)
(5, 17635)
(8, 17635)
(6, 17635)
(11, 17635)
(2, 17635)
(2, 17635)
(3, 17635)
(6, 17635)
(5, 17635)
(4, 17635)
(4, 17635)
(6, 17635)
(11, 17635)
(8, 17635)
(5, 17635)
(6, 17635)
(8, 17635)
(9, 17635)
(4, 17635)
(12, 17635)
(10, 17635)
(4, 17635)
(6, 17635)
(4, 17635)
(13, 17635)
(8, 17635)
(9, 17635)
(4, 17635)
(6, 17635)
(15, 17635)
(2, 17635)
(8, 17635)
(9, 17635)
(7, 17635)
(5, 17635)
(12, 17635)
(6, 17635)
(4, 17635)
(9, 17635)
(12, 17635)
(7, 17635)
(8, 17635)
(5, 17635)
(16, 17635)
(12, 17635)
(12, 17635)
(10, 17635)
(2, 17635)
(4, 17635)
(19, 17635)
(4, 17635)
(9, 17635)
(11, 17635)
(11, 17635)
(5, 17635)
(7, 17635)
(2, 17635)
(8, 17635)
(2, 17635)
(7, 17635)
(2, 17635)
(6, 17635)
(6, 17635)
(9, 17635)
(4, 17635)
(4, 17635)
(15, 17635)
(4, 17635)
(12, 17635)
(6, 17635)
(7, 17635)
(4, 17635

(10, 17635)
(4, 17635)
(25, 17635)
(2, 17635)
(18, 17635)
(3, 17635)
(4, 17635)
(4, 17635)
(6, 17635)
(4, 17635)
(11, 17635)
(4, 17635)
(8, 17635)
(5, 17635)
(20, 17635)
(10, 17635)
(6, 17635)
(5, 17635)
(8, 17635)
(4, 17635)
(8, 17635)
(6, 17635)
(4, 17635)
(10, 17635)
(2, 17635)
(4, 17635)
(6, 17635)
(4, 17635)
(8, 17635)
(10, 17635)
(4, 17635)
(11, 17635)
(7, 17635)
(4, 17635)
(14, 17635)
(2, 17635)
(8, 17635)
(4, 17635)
(10, 17635)
(11, 17635)
(3, 17635)
(4, 17635)
(8, 17635)
(12, 17635)
(10, 17635)
(6, 17635)
(9, 17635)
(12, 17635)
(6, 17635)
(4, 17635)
(4, 17635)
(15, 17635)
(4, 17635)
(4, 17635)
(9, 17635)
(12, 17635)
(9, 17635)
(4, 17635)
(5, 17635)
(4, 17635)
(11, 17635)
(7, 17635)
(5, 17635)
(3, 17635)
(16, 17635)
(2, 17635)
(9, 17635)
(7, 17635)
(14, 17635)
(4, 17635)
(10, 17635)
(3, 17635)
(6, 17635)
(4, 17635)
(14, 17635)
(4, 17635)
(4, 17635)
(10, 17635)
(7, 17635)
(12, 17635)
(12, 17635)
(4, 17635)
(6, 17635)
(5, 17635)
(2, 17635)
(15, 17635)
(9, 17635)
(2, 17635)
(12, 1

(6, 17635)
(6, 17635)
(9, 17635)
(6, 17635)
(8, 17635)
(3, 17635)
(8, 17635)
(4, 17635)
(10, 17635)
(4, 17635)
(8, 17635)
(6, 17635)
(11, 17635)
(4, 17635)
(7, 17635)
(4, 17635)
(5, 17635)
(4, 17635)
(6, 17635)
(6, 17635)
(18, 17635)
(8, 17635)
(9, 17635)
(8, 17635)
(18, 17635)
(10, 17635)
(13, 17635)
(4, 17635)
(2, 17635)
(2, 17635)
(9, 17635)
(7, 17635)
(6, 17635)
(4, 17635)
(6, 17635)
(4, 17635)
(7, 17635)
(10, 17635)
(11, 17635)
(4, 17635)
(7, 17635)
(15, 17635)
(4, 17635)
(4, 17635)
(9, 17635)
(4, 17635)
(7, 17635)
(6, 17635)
(8, 17635)
(8, 17635)
(5, 17635)
(5, 17635)
(10, 17635)
(4, 17635)
(2, 17635)
(20, 17635)
(12, 17635)
(6, 17635)
(5, 17635)
(3, 17635)
(3, 17635)
(4, 17635)
(10, 17635)
(9, 17635)
(3, 17635)
(5, 17635)
(5, 17635)
(4, 17635)
(10, 17635)
(10, 17635)
(13, 17635)
(8, 17635)
(7, 17635)
(3, 17635)
(8, 17635)
(19, 17635)
(7, 17635)
(2, 17635)
(11, 17635)
(4, 17635)
(2, 17635)
(18, 17635)
(9, 17635)
(4, 17635)
(4, 17635)
(3, 17635)
(8, 17635)
(5, 17635)
(17, 17635)
(

(4, 17635)
(6, 17635)
(8, 17635)
(2, 17635)
(10, 17635)
(9, 17635)
(11, 17635)
(4, 17635)
(2, 17635)
(4, 17635)
(13, 17635)
(12, 17635)
(13, 17635)
(12, 17635)
(6, 17635)
(10, 17635)
(5, 17635)
(12, 17635)
(4, 17635)
(10, 17635)
(10, 17635)
(23, 17635)
(14, 17635)
(9, 17635)
(7, 17635)
(13, 17635)
(11, 17635)
(8, 17635)
(13, 17635)
(5, 17635)
(10, 17635)
(11, 17635)
(8, 17635)
(4, 17635)
(4, 17635)
(17, 17635)
(7, 17635)
(5, 17635)
(5, 17635)
(2, 17635)
(5, 17635)
(8, 17635)
(5, 17635)
(9, 17635)
(5, 17635)
(4, 17635)
(15, 17635)
(14, 17635)
(5, 17635)
(8, 17635)
(9, 17635)
(9, 17635)
(4, 17635)
(3, 17635)
(14, 17635)
(8, 17635)
(7, 17635)
(6, 17635)
(3, 17635)
(5, 17635)
(6, 17635)
(6, 17635)
(2, 17635)
(15, 17635)
(10, 17635)
(11, 17635)
(12, 17635)
(14, 17635)
(10, 17635)
(15, 17635)
(11, 17635)
(12, 17635)
(9, 17635)
(6, 17635)
(10, 17635)
(8, 17635)
(4, 17635)
(4, 17635)
(12, 17635)
(4, 17635)
(4, 17635)
(4, 17635)
(6, 17635)
(8, 17635)
(6, 17635)
(13, 17635)
(12, 17635)
(4, 17635

(7, 17635)
(2, 17635)
(4, 17635)
(3, 17635)
(4, 17635)
(5, 17635)
(8, 17635)
(10, 17635)
(6, 17635)
(4, 17635)
(9, 17635)
(11, 17635)
(7, 17635)
(7, 17635)
(3, 17635)
(5, 17635)
(6, 17635)
(11, 17635)
(6, 17635)
(4, 17635)
(4, 17635)
(2, 17635)
(9, 17635)
(8, 17635)
(9, 17635)
(6, 17635)
(11, 17635)
(6, 17635)
(4, 17635)
(14, 17635)
(12, 17635)
(3, 17635)
(4, 17635)
(9, 17635)
(12, 17635)
(6, 17635)
(17, 17635)
(7, 17635)
(4, 17635)
(8, 17635)
(4, 17635)
(9, 17635)
(12, 17635)
(5, 17635)
(6, 17635)
(9, 17635)
(5, 17635)
(2, 17635)
(3, 17635)
(10, 17635)
(4, 17635)
(23, 17635)
(2, 17635)
(4, 17635)
(10, 17635)
(9, 17635)
(13, 17635)
(4, 17635)
(10, 17635)
(5, 17635)
(9, 17635)
(5, 17635)
(4, 17635)
(2, 17635)
(4, 17635)
(3, 17635)
(13, 17635)
(12, 17635)
(8, 17635)
(4, 17635)
(4, 17635)
(4, 17635)
(10, 17635)
(8, 17635)
(4, 17635)
(9, 17635)
(4, 17635)
(9, 17635)
(9, 17635)
(7, 17635)
(13, 17635)
(3, 17635)
(6, 17635)
(9, 17635)
(4, 17635)
(10, 17635)
(4, 17635)
(4, 17635)
(10, 17635)
(

...

(6, 17635)
(4, 17635)
(7, 17635)
(10, 17635)
(7, 17635)
(12, 17635)
(8, 17635)
(17, 17635)
(13, 17635)
(6, 17635)
(8, 17635)
(10, 17635)
(7, 17635)
(4, 17635)
(10, 17635)
(5, 17635)
(6, 17635)
(9, 17635)
(8, 17635)
(8, 17635)
(10, 17635)
(9, 17635)
(7, 17635)
(26, 17635)
(12, 17635)
(5, 17635)
(9, 17635)
(11, 17635)
(6, 17635)
(5, 17635)
(6, 17635)
(3, 17635)
(6, 17635)
(3, 17635)
(9, 17635)
(6, 17635)
(2, 17635)
(11, 17635)
(12, 17635)
(5, 17635)
(4, 17635)
(6, 17635)
(10, 17635)
(12, 17635)
(7, 17635)
(8, 17635)
(14, 17635)
(13, 17635)
(8, 17635)
(10, 17635)
(4, 17635)
(4, 17635)
(5, 17635)
(12, 17635)
(2, 17635)
(8, 17635)
(4, 17635)
(3, 17635)
(7, 17635)
(7, 17635)
(2, 17635)
(4, 17635)
(12, 17635)
(5, 17635)
(9, 17635)
(11, 17635)
(6, 17635)
(5, 17635)
(9, 17635)
(4, 17635)
(4, 17635)
(5, 17635)
(10, 17635)
(4, 17635)
(8, 17635)
(5, 17635)
(11, 17635)
(8, 17635)
(4, 17635)
(4, 17635)
(10, 17635)
(5, 17635)
(10, 17635)
(5, 17635)
(8, 17635)
(8, 17635)
(6, 17635)
(7, 17635)
(4, 1763

(4, 17635)
(11, 17635)
(12, 17635)
(5, 17635)
(7, 17635)
(10, 17635)
(2, 17635)
(4, 17635)
(9, 17635)
(12, 17635)
(5, 17635)
(7, 17635)
(14, 17635)
(8, 17635)
(12, 17635)
(13, 17635)
(10, 17635)
(4, 17635)
(9, 17635)
(4, 17635)
(7, 17635)
(9, 17635)
(3, 17635)
(10, 17635)
(12, 17635)
(5, 17635)
(12, 17635)
(6, 17635)
(4, 17635)
(4, 17635)
(3, 17635)
(12, 17635)
(9, 17635)
(4, 17635)
(8, 17635)
(8, 17635)
(10, 17635)
(5, 17635)
(11, 17635)
(4, 17635)
(8, 17635)
(12, 17635)
(11, 17635)
(10, 17635)
(5, 17635)
(14, 17635)
(7, 17635)
(6, 17635)
(4, 17635)
(12, 17635)
(15, 17635)
(8, 17635)
(12, 17635)
(12, 17635)
(5, 17635)
(12, 17635)
(4, 17635)
(4, 17635)
(13, 17635)
(5, 17635)
(3, 17635)
(7, 17635)
(8, 17635)
(2, 17635)
(8, 17635)
(5, 17635)
(17, 17635)
(14, 17635)
(8, 17635)
(9, 17635)
(11, 17635)
(7, 17635)
(5, 17635)
(7, 17635)
(15, 17635)
(12, 17635)
(4, 17635)
(12, 17635)
(6, 17635)
(5, 17635)
(12, 17635)
(12, 17635)
(9, 17635)
(5, 17635)
(10, 17635)
(7, 17635)
(4, 17635)
(2, 17635)

(5, 17635)
(3, 17635)
(12, 17635)
(6, 17635)
(12, 17635)
(12, 17635)
(4, 17635)
(4, 17635)
(8, 17635)
(6, 17635)
(4, 17635)
(6, 17635)
(16, 17635)
(11, 17635)
(12, 17635)
(10, 17635)
(15, 17635)
(14, 17635)
(8, 17635)
(10, 17635)
(4, 17635)
(10, 17635)
(6, 17635)
(6, 17635)
(8, 17635)
(6, 17635)
(8, 17635)
(4, 17635)
(4, 17635)
(5, 17635)
(12, 17635)
(6, 17635)
(16, 17635)
(7, 17635)
(4, 17635)
(6, 17635)
(6, 17635)
(10, 17635)
(8, 17635)
(17, 17635)
(12, 17635)
(12, 17635)
(7, 17635)
(11, 17635)
(10, 17635)
(4, 17635)
(12, 17635)
(6, 17635)
(6, 17635)
(5, 17635)
(3, 17635)
(10, 17635)
(15, 17635)
(14, 17635)
(16, 17635)
(8, 17635)
(20, 17635)
(9, 17635)
(10, 17635)
(4, 17635)
(10, 17635)
(2, 17635)
(10, 17635)
(12, 17635)
(14, 17635)
(10, 17635)
(8, 17635)
(12, 17635)
(8, 17635)
(8, 17635)
(7, 17635)
(5, 17635)
(12, 17635)
(11, 17635)
(7, 17635)
(5, 17635)
(5, 17635)
(9, 17635)
(6, 17635)
(2, 17635)
(10, 17635)
(12, 17635)
(9, 17635)
(7, 17635)
(9, 17635)
(10, 17635)
(7, 17635)
(4, 17

(8, 17635)
(4, 17635)
(8, 17635)
(9, 17635)
(7, 17635)
(17, 17635)
(7, 17635)
(7, 17635)
(12, 17635)
(3, 17635)
(5, 17635)
(4, 17635)
(7, 17635)
(4, 17635)
(4, 17635)
(4, 17635)
(12, 17635)
(12, 17635)
(17, 17635)
(6, 17635)
(5, 17635)
(6, 17635)
(10, 17635)
(6, 17635)
(6, 17635)
(10, 17635)
(9, 17635)
(2, 17635)
(5, 17635)
(9, 17635)
(10, 17635)
(12, 17635)
(4, 17635)
(12, 17635)
(12, 17635)
(4, 17635)
(13, 17635)
(8, 17635)
(4, 17635)
(7, 17635)
(4, 17635)
(11, 17635)
(4, 17635)
(12, 17635)
(12, 17635)
(8, 17635)
(6, 17635)
(4, 17635)
(3, 17635)
(2, 17635)
(10, 17635)
(5, 17635)
(7, 17635)
(3, 17635)
(10, 17635)
(12, 17635)
(4, 17635)
(4, 17635)
(4, 17635)
(12, 17635)
(4, 17635)
(14, 17635)
(12, 17635)
(7, 17635)
(12, 17635)
(10, 17635)
(7, 17635)
(12, 17635)
(12, 17635)
(5, 17635)
(7, 17635)
(10, 17635)
(18, 17635)
(6, 17635)
(12, 17635)
(12, 17635)
(4, 17635)
(18, 17635)
(6, 17635)
(4, 17635)
(8, 17635)
(5, 17635)
(12, 17635)
(7, 17635)
(6, 17635)
(11, 17635)
(12, 17635)
(12, 17635

(5, 17635)
(10, 17635)
(10, 17635)
(4, 17635)
(13, 17635)
(6, 17635)
(13, 17635)
(6, 17635)
(7, 17635)
(8, 17635)
(11, 17635)
(4, 17635)
(7, 17635)
(3, 17635)
(8, 17635)
(2, 17635)
(6, 17635)
(4, 17635)
(13, 17635)
(12, 17635)
(8, 17635)
(8, 17635)
(4, 17635)
(6, 17635)
(14, 17635)
(7, 17635)
(14, 17635)
(6, 17635)
(4, 17635)
(4, 17635)
(6, 17635)
(8, 17635)
(6, 17635)
(11, 17635)
(5, 17635)
(3, 17635)
(12, 17635)
(10, 17635)
(4, 17635)
(9, 17635)
(7, 17635)
(16, 17635)
(6, 17635)
(11, 17635)
(8, 17635)
(4, 17635)
(7, 17635)
(23, 17635)
(8, 17635)
(10, 17635)
(12, 17635)
(5, 17635)
(9, 17635)
(12, 17635)
(9, 17635)
(11, 17635)
(4, 17635)
(18, 17635)
(3, 17635)
(9, 17635)
(3, 17635)
(7, 17635)
(5, 17635)
(4, 17635)
(16, 17635)
(3, 17635)
(12, 17635)
(5, 17635)
(6, 17635)
(6, 17635)
(11, 17635)
(5, 17635)
(9, 17635)
(5, 17635)
(4, 17635)
(11, 17635)
(10, 17635)
(6, 17635)
(9, 17635)
(10, 17635)
(8, 17635)
(6, 17635)
(14, 17635)
(16, 17635)
(4, 17635)
(8, 17635)
(12, 17635)
(8, 17635)
(9,

In [23]:
x_train_new = []
for r in x_test:
    print(r.shape)
    p1 = model1.predict_proba(r)
#     print(p1.shape)
    p2 = model2.predict_proba(r)
    res = np.concatenate((p1, p2), 1)
    p3 = model3.predict_proba(r)
    res = np.concatenate((res, p3), 1)
    x_train_new.append(res)
x_test = x_train_new

(7, 17635)
(10, 17635)
(4, 17635)
(16, 17635)
(8, 17635)
(4, 17635)
(8, 17635)
(12, 17635)
(12, 17635)
(7, 17635)
(13, 17635)
(5, 17635)
(13, 17635)
(7, 17635)
(14, 17635)
(7, 17635)
(8, 17635)
(11, 17635)
(10, 17635)
(4, 17635)
(12, 17635)
(11, 17635)
(8, 17635)
(6, 17635)
(9, 17635)
(8, 17635)
(14, 17635)
(7, 17635)
(7, 17635)
(18, 17635)
(10, 17635)
(7, 17635)
(7, 17635)
(7, 17635)
(10, 17635)
(7, 17635)
(5, 17635)
(9, 17635)
(4, 17635)
(10, 17635)
(11, 17635)
(2, 17635)
(8, 17635)
(7, 17635)
(16, 17635)
(5, 17635)
(7, 17635)
(10, 17635)
(10, 17635)
(12, 17635)
(10, 17635)
(5, 17635)
(3, 17635)
(4, 17635)
(7, 17635)
(9, 17635)
(7, 17635)
(23, 17635)
(7, 17635)
(2, 17635)
(14, 17635)
(11, 17635)
(9, 17635)
(3, 17635)
(13, 17635)
(7, 17635)
(8, 17635)
(4, 17635)
(2, 17635)
(7, 17635)
(12, 17635)
(9, 17635)
(2, 17635)
(4, 17635)
(6, 17635)
(14, 17635)
(20, 17635)
(13, 17635)
(10, 17635)
(5, 17635)
(4, 17635)
(11, 17635)
(22, 17635)
(4, 17635)
(8, 17635)
(8, 17635)
(10, 17635)
(8, 17635

(4, 17635)
(9, 17635)
(16, 17635)
(2, 17635)
(9, 17635)
(4, 17635)
(10, 17635)
(7, 17635)
(10, 17635)
(9, 17635)
(5, 17635)
(8, 17635)
(8, 17635)
(7, 17635)
(6, 17635)
(6, 17635)
(16, 17635)
(6, 17635)
(7, 17635)
(6, 17635)
(13, 17635)
(12, 17635)
(4, 17635)
(8, 17635)
(5, 17635)
(12, 17635)
(5, 17635)
(4, 17635)
(7, 17635)
(6, 17635)
(2, 17635)
(5, 17635)
(10, 17635)
(4, 17635)
(15, 17635)
(4, 17635)
(12, 17635)
(8, 17635)
(10, 17635)
(7, 17635)
(6, 17635)
(6, 17635)
(8, 17635)
(9, 17635)
(4, 17635)
(8, 17635)
(12, 17635)
(11, 17635)
(8, 17635)
(9, 17635)
(8, 17635)
(2, 17635)
(11, 17635)
(12, 17635)
(4, 17635)
(8, 17635)
(13, 17635)
(7, 17635)
(12, 17635)
(8, 17635)
(13, 17635)
(2, 17635)
(6, 17635)
(4, 17635)
(4, 17635)
(9, 17635)
(7, 17635)
(6, 17635)
(4, 17635)
(10, 17635)
(8, 17635)
(4, 17635)
(4, 17635)
(14, 17635)
(3, 17635)
(4, 17635)
(6, 17635)
(16, 17635)
(4, 17635)
(6, 17635)
(14, 17635)
(6, 17635)
(2, 17635)
(8, 17635)
(6, 17635)
(6, 17635)
(11, 17635)
(15, 17635)
(16, 176

In [24]:
learning_rate = 0.1
#training_epochs = 7
input_size = (x_train[0].shape)[1]
#batch_size = 25
#num_units = 128 # the number of units in the LSTM cell
number_of_classes = len(set(trainY_all))

In [ ]:
number_of_classes

In [ ]:
x_train[0].shape

In [ ]:
input_size = 12

In [25]:
def pad_data(sentence, max_length, input_size):
#     sentence = sentence.toarray()
    pad_len = max_length - len(sentence)
    temp = sentence
    if pad_len > 0:
        single_pad = np.zeros(input_size)
        padding = [single_pad] * pad_len
        temp = np.concatenate((sentence, padding))
    return temp

In [26]:
input_size

12

In [27]:
def pad_label(sentence, max_length):
    pad_len = max_length - len(sentence)
    padding = np.zeros(pad_len, dtype=int)
    return np.concatenate((sentence, padding))

In [28]:
def batch(data, labels, sequence_lengths, batch_size, input_size):
    n_batch = int(math.ceil(float(len(data)) / (batch_size)))
    index = 0
    for _ in range(n_batch):
        batch_sequence_lengths = np.array(sequence_lengths[index: index + batch_size])
        batch_length = np.array(max(batch_sequence_lengths)) # max length in batch
        batch_data = np.array([pad_data(x, batch_length, input_size) for x in data[index: index + batch_size]]) # pad data
        batch_labels = np.array([pad_label(x, batch_length) for x in labels[index: index + batch_size]]) # pad labels
        index += batch_size
        
        # Reshape input data to be suitable for LSTMs.
        batch_data = batch_data.reshape(-1, batch_length, input_size)
        
        yield batch_data, batch_labels, batch_length, batch_sequence_lengths

In [29]:
input_data = tf.placeholder(tf.float32, [None, None, input_size], name="input_data") # shape = (batch, batch_seq_len, input_size)
labels = tf.placeholder(tf.int32, shape=[None, None], name="labels") # shape = (batch, sentence)
batch_sequence_length = tf.placeholder(tf.int32) # max sequence length in batch
original_sequence_lengths = tf.placeholder(tf.int32, [None])


In [30]:
weights = tf.get_variable("weights", [input_size, number_of_classes])
matricized_x_train = tf.reshape(input_data, [-1, input_size])
matricized_unary_scores = tf.matmul(matricized_x_train, weights)
#matricized_unary_scores = tf.sigmoid(matricized_unary_scores_temp)
scores = tf.reshape(matricized_unary_scores, [-1, batch_sequence_length, number_of_classes])
#scores = tf.reshape(matricized_unary_scores, [num_examples, num_words, num_tags])


In [ ]:
# input_data = tf.placeholder(tf.float32, [None, None])

In [31]:
# Linear-CRF.
log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(scores, labels, original_sequence_lengths)

loss = tf.reduce_mean(-log_likelihood)

# Compute the viterbi sequence and score (used for prediction and test time).
viterbi_sequence, viterbi_score = tf.contrib.crf.crf_decode(scores, transition_params, original_sequence_lengths)

# Training ops.
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss)

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [32]:
training_epochs = 5;
batch_size = 500
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for i in range(training_epochs):
        for batch_data, batch_labels, batch_seq_len, batch_sequence_lengths in batch(x_train, y_train, sequence_length_train, batch_size, input_size):
            tf_viterbi_sequence, _ = session.run([viterbi_sequence, train_op], 
                                                 feed_dict={input_data: batch_data, 
                                                            labels: batch_labels, 
                                                            batch_sequence_length: batch_seq_len,
                                                            original_sequence_lengths: batch_sequence_lengths })
            # Show train accuracy.
            if i % 1 == 0:
                # Create a mask to fix input lengths.
                mask = (np.expand_dims(np.arange(batch_seq_len), axis=0) <
                    np.expand_dims(batch_sequence_lengths, axis=1))
                total_labels = np.sum(batch_sequence_lengths)
                correct_labels = np.sum((batch_labels == tf_viterbi_sequence) * mask)
                accuracy = 100.0 * correct_labels / float(total_labels)
                print("Epoch: %d" % i, "Accuracy: %.2f%%" % accuracy)
    
    # Save the variables to disk.
    saver.save(session, MODEL_PATH)

Epoch: 0 Accuracy: 11.16%
Epoch: 0 Accuracy: 58.10%
Epoch: 0 Accuracy: 66.65%
Epoch: 0 Accuracy: 77.18%
Epoch: 0 Accuracy: 79.35%
Epoch: 0 Accuracy: 77.16%
Epoch: 0 Accuracy: 83.23%
Epoch: 0 Accuracy: 84.05%
Epoch: 0 Accuracy: 86.71%
Epoch: 0 Accuracy: 86.38%
Epoch: 0 Accuracy: 87.69%
Epoch: 0 Accuracy: 89.70%
Epoch: 0 Accuracy: 90.63%
Epoch: 0 Accuracy: 92.49%
Epoch: 0 Accuracy: 92.31%
Epoch: 0 Accuracy: 92.53%
Epoch: 0 Accuracy: 92.88%
Epoch: 0 Accuracy: 92.46%
Epoch: 0 Accuracy: 92.95%
Epoch: 0 Accuracy: 92.79%
Epoch: 0 Accuracy: 93.90%
Epoch: 0 Accuracy: 94.00%
Epoch: 0 Accuracy: 95.10%
Epoch: 1 Accuracy: 95.17%
Epoch: 1 Accuracy: 94.94%
Epoch: 1 Accuracy: 95.16%
Epoch: 1 Accuracy: 95.45%
Epoch: 1 Accuracy: 94.33%
Epoch: 1 Accuracy: 94.69%
Epoch: 1 Accuracy: 95.26%
Epoch: 1 Accuracy: 96.01%
Epoch: 1 Accuracy: 95.18%
Epoch: 1 Accuracy: 94.77%
Epoch: 1 Accuracy: 95.34%
Epoch: 1 Accuracy: 95.63%
Epoch: 1 Accuracy: 95.35%
Epoch: 1 Accuracy: 95.82%
Epoch: 1 Accuracy: 96.08%
Epoch: 1 Acc

In [33]:
# Testing the model.
with tf.Session() as session:
    # Restore variables from disk.
    saver.restore(session, MODEL_PATH)
    
    for batch_data, batch_labels, batch_seq_len, batch_sequence_lengths in batch(x_test, y_test, sequence_length_test, len(x_test), input_size):
        tf_viterbi_sequence = session.run(viterbi_sequence, feed_dict={input_data: batch_data, 
                                                                       labels: batch_labels, 
                                                                       batch_sequence_length: batch_seq_len,
                                                                       original_sequence_lengths: batch_sequence_lengths })
    # mask to correct input sizes
    mask = (np.expand_dims(np.arange(batch_seq_len), axis=0) <
        np.expand_dims(batch_sequence_lengths, axis=1))
    total_labels = np.sum(batch_sequence_lengths)
    correct_labels = np.sum((batch_labels == tf_viterbi_sequence) * mask)
    accuracy = 100.0 * correct_labels / float(total_labels)
    print("Test accuracy: %.2f%%" % accuracy)

    print("Label:", batch_labels[0].astype(int))    
    print("Pred.:", tf_viterbi_sequence[0])

INFO:tensorflow:Restoring parameters from ./model_crf_v1.ckpt
Test accuracy: 75.52%
Label: [1 0 2 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Pred.: [1 0 2 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


# Evaluation

In [ ]:
from sklearn import metrics

In [ ]:
preds = []
for i in range(len(sequence_length_test)):
    preds.extend(tf_viterbi_sequence[i][:sequence_length_test[i]])

In [ ]:
metrics.accuracy_score(valY_all, preds)

In [ ]:
print(metrics.classification_report(valY_all, preds))

In [ ]:
cmat = metrics.confusion_matrix(valY_all, preds)

In [ ]:
cmat.diagonal()/cmat.sum(axis=1)

In [ ]:
cmat

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
classes = ['inform', 'question', 'directive', 'commisive']

In [ ]:
plot_confusion_matrix(cmat, classes)

In [ ]:
plot_confusion_matrix(cmat, classes, normalize=True)

In [ ]:
correctPerClass = {}
totalPerClass   = {}
for cls in [0, 1, 2, 3]:
    correctPerClass[cls] = 0
    totalPerClass[cls]   = 0
for yt, yp in zip(valY_all, preds):
    totalPerClass[yt] += 1
    correctPerClass[yt] += int(yt==yp)



In [ ]:
totalPerClass

In [ ]:
correctPerClass

In [ ]:
for cls in [0, 1, 2, 3]:
    acc = float(correctPerClass[cls])/totalPerClass[cls]
    print("Accuracy for class ", cls, " :", acc)

# Saving the transition params

In [ ]:
x = tf.Variable([[1.0, 2.0], [3, 1]])

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
v = sess.run(transition_params)    

In [ ]:
np.savetxt('transition_params_crf_v1.csv', v, delimiter=',')